## Without web search

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=0.0)

agent = create_agent(
    model=model,
)

In [3]:
from langchain.messages import HumanMessage

question = HumanMessage(content="How up to date is your training knowledge?")

response = agent.invoke(
    {"messages": [question]}
)

In [4]:
print(response['messages'][-1].content)

[{'type': 'text', 'text': 'My knowledge cutoff is **January 2024**.\n\nThis means that my internal training data includes information up to that point. However, I have a few ways to provide more current information:\n\n1.  **Google Search:** I can access Google Search in real-time to look up current events, news, and developments that have happened since my last major update.\n2.  **Continuous Updates:** While I have a specific "cutoff" for my primary training set, I am part of a system that receives regular updates and refinements.\n\n**In short:** If you ask about a historical event or a general concept, I rely on my training data. If you ask about something that happened this morning or last week, I will likely use Google Search to give you an accurate answer.', 'extras': {'signature': 'EocNCoQNAXLI2nyKKlqfhXCNQN0I72hnn/Fh1Fdv6MQnmr7HZBrXS7D7XFhI5UuqgVuuT3NZO1KwNUCbxgQ4el7be75ZuMoxhinIpCG1NNKToou/sJ2XzrtOsQUfZoTUdqtfL9RXVa2IU3Rx5pmNRjObFaF8izDmXOJ3OT/BLNbRNdzF8fjRegK8ywQiM7+dCvlQ3WS

In [5]:
question = HumanMessage(content="Who is the mayor of San Francisco?")
response = agent.invoke(
    {"messages": [question]}
)
print(response['messages'][-1].content)

[{'type': 'text', 'text': 'The mayor of San Francisco is **London Breed**. She has been in office since July 2018.', 'extras': {'signature': 'EqQCCqECAXLI2nyEsDiDGEV5Q9OF/iMQpbnWDv1ShuTLu6TDaVNuVxFpduWpnw82DIRwnG+rTnSXwxMBAPS0U+vJnNjfJj/rVbL/lfV7B/UJcIwu+sAaW6u1R7VbBI6l8Q4bog9ZrDOFP9p1u/1kOgoHnKaZCdmMn+7BtMelcuoQIKF3M1b1yi9pNRZH4OCTi9+hjvOGjuAD7+etX/GlI+80n3CMKsGYr7I678WtLWUC5+BrWv+sL0vQVBWi2XCfHHa+tamPXwdZMjfvtzsDvyR3EyrpqFVwFZhSclTWv2fwgoeIsamPt9VfddbolygroXor+yYNKkvR3dCLwpRwLjRleHkN+jTl2dKmtUT8ytWTpeMLp+tYIR49thFUW6K7Gv2W0lYwag=='}}]


## Add web search tool

In [6]:
from langchain.tools import tool
from typing import Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

web_search.invoke("Who is the current mayor of San Francisco?")

{'query': 'Who is the current mayor of San Francisco?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Mayor_of_San_Francisco',
   'title': 'Mayor of San Francisco - Wikipedia',
   'content': 'The current mayor is Democrat Daniel Lurie.',
   'score': 0.95324475,
   'raw_content': None},
  {'url': 'https://apnews.com/article/san-francisco-new-mayor-liberal-city-81ea0a7b37af6cbb68aea7ef5cc6a4f0',
   'title': "San Francisco's new mayor is starting to unite the fractured city",
   'content': 'San Francisco Mayor Daniel Lurie, a political newcomer and Levi Strauss heir, has marked his first 100 days with a hands-on, business-friendly approach.',
   'score': 0.8755427,
   'raw_content': None},
  {'url': 'https://www.sf.gov/departments--office-mayor',
   'title': 'Office of the Mayor - SF.gov',
   'content': 'Daniel Lurie is the 46th Mayor of the City and County of San Francisco.',
   'score': 0.8488849,
   'raw_content': None

In [7]:
agent = create_agent(
    model=model,
    tools=[web_search]
)

question = HumanMessage(content="Who is the current mayor of San Francisco?")

response = agent.invoke(
    {"messages": [question]}
)
print(response['messages'][-1].content)

[{'type': 'text', 'text': "The current mayor of San Francisco is **Daniel Lurie**. \n\nHe was sworn in as the city's 46th mayor on January 8, 2025, after winning the November 2024 election. He succeeded London Breed. Lurie is a philanthropist and an heir to the Levi Strauss & Co. fortune, and he is the first person in over a century to be elected mayor of San Francisco without prior experience in government or elective office.", 'extras': {'signature': 'ErUHCrIHAXLI2nxP9T5q+iID1Pg3kL7714gMccUZRclgXpuUCsxy9vXzKNn2ekjSSm0xpXJyY3dHDE45alUMNtseFlsFRdR35BsNfb+C0KGXHX71xYtT2ISDwRPZQ53kDCRRPuWl4ocCXT3D69BXU8oEEvAn/Wh1GLOjORzpyqTFoiVCs5zEfIT0ZWWFis2xCaYb97P465rdzU0eXkFIv8hOXO7eFfyj19OU20jLwcAEg8ZyuaaVGDoplDQWrz+KNYEcUmClE2JFOeNj/3geS6H8OqBPc928QTK4x9FZQLwA5TpkqIHnjaHAs4hvM41Lrtl25ZhnU3HVyFjREh4N8TDnc7J+HK6mlzX5vxBnH/H2IV7y3cykff+eLC/7QLfPuCZyb0T7W6oFX68RcOeTsZfoYdC++8WnGtAsSS0776EAGpqxst3xo4yUhCWUapr932R/DBRZT8G2nfNlW1QwQbQhkHZoi4zV00hI2DTO7lPonNcGByqS5s6yuOCJIMdVvFS0K7D41y54cqmlgMqXXoNj0fB8IM

In [8]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content='Who is the current mayor of San Francisco?', additional_kwargs={}, response_metadata={}, id='b178a033-6cc9-4dec-ba97-8f95745fe71a'),
 AIMessage(content=[], additional_kwargs={'function_call': {'name': 'web_search', 'arguments': '{"query": "current mayor of San Francisco"}'}, '__gemini_function_call_thought_signatures__': {'a4dd7f90-c9da-4a92-b599-368f23e1a627': 'EswBCskBAXLI2nxcoKM6/nbFIWyVmK7kJLQydGwGm96/2OXc2vjKZJTYvcdmA9zoSRwp3oB4TkwZRP7OSWQfSD6CrB3wxzC8sNvwYhsTLIMF0Jrridjt4vsItjCtHeAuQWshcsCWthnPQn0BonIsDjVAaYU6Tbi8RqBWgkrpFhU3tQgWaUoPR54PXK9BzVzCGSEZ6rG9ioIgPRrFQ6xfDuyXyDpiaBqG3p9H66MfJJHA7i38Kuf2Ps8LoNb39zq/YV9FcOz/v8Ql35Rb629x'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bf5bb-6bc7-7840-ae4d-fd58f06a7285-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'current mayor of San Francisco'}, 'id': 'a4dd7f90-c9da-4a92-b599-368f23e1a627', 'ty

Trace: https://smith.langchain.com/public/e8eb5102-6180-4baf-8eb7-c0b3cee78525/r